In [2]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=49135e10149411f58c5b9ea7af2a391e5498b71814270454788c98a9a59a006f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langdetect import detect
import ipywidgets as widgets
from IPython.display import display, clear_output

In [4]:
model_name = 'facebook/nllb-200-distilled-600M'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Language mapping
lang_map = {
    'en': 'eng_Latn',
    'hi': 'hin_Deva',
    'es': 'spa_Latn',
    'fr': 'fra_Latn',
    'de': 'deu_Latn',
    'mai': 'mai_Deva',
    'bn': 'ben_Beng',  # Bengali
    'ta': 'tam_Taml',  # Tamil
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [5]:
text_input = widgets.Textarea(
    value='Welcome to India! Let us celebrate together.',
    description='Text:',
    layout=widgets.Layout(width='100%', height='80px')
)

# Language selection
lang_dropdown = widgets.Dropdown(
    options=[
        ('Hindi', 'hin_Deva'),
        ('Maithili', 'mai_Deva'),
        ('Bengali', 'ben_Beng'),
        ('Tamil', 'tam_Taml')
    ],
    value='hin_Deva',
    description='To:'
)

# Output widget
output = widgets.Output()

# Translate function
def translate(input_text, src_lang=None, target_lang='hin_Deva'):
    if src_lang is None:
        detected = detect(input_text)
        src_lang = lang_map.get(detected, 'eng_Latn')
    translator = pipeline('translation', model=model, tokenizer=tokenizer,
                          src_lang=src_lang, tgt_lang=target_lang, max_length=400)
    result = translator(input_text)[0]['translation_text']
    return result

# On button click
def on_button_clicked(b):
    output.clear_output()
    with output:
        result = translate(text_input.value, target_lang=lang_dropdown.value)
        print(f"Translated to {lang_dropdown.label}:\n{result}")

# Button
translate_button = widgets.Button(description="Translate", button_style='info')
translate_button.on_click(on_button_clicked)

# Display UI
display(widgets.VBox([text_input, lang_dropdown, translate_button, output]))
